In [ ]:
import spacy
from spacy import displacy
import random
import json
import logging
import re

In [ ]:
with open('annotations-1.json', 'r') as file:
    json_data = json.load(file)
classes = json_data['classes']
annotations = json_data['annotations']

In [ ]:
annotations = [annotation for annotation in annotations if annotation is not None]

# Print the remaining annotations
annotations

[["\r\nyear old man 1 month s/p AVR, MVR, and CABG x 3, now with\r\nfever to 100.7, WBC 18.7, and erythema at superior aspect of\r\nwound. Pt recently had a long hospitalization following CABG and\r\nMVR/AVR([**Date range (3) 83151**]), which was complicated by post-op\r\nCVA, aflutter/afib with unsuccessful cardioversion, and a\r\nHA-pna. The pt was discharged with a trach to [**Hospital3 **],\r\ndue to his inability to manage secretions. The pt did well at\r\nrehab, until [**2125-10-4**], when he developed a low-grade fever and\r\nCXR showed a new left lung base consolidation. He continued to\r\nhave temps to 99, and on [**10-14**] pt was started on ceftriaxone 1g\r\nq24h. On [**10-15**] the pt was also started on vancomycin 1g q12h. Per\r\nrehab reports the pt developed confusion and RR in the high\r\nThe pt was thought to be in volume overload, and was given\r\nlasix 20mg IV once on [**10-17**]. Gram stain of sputum from [**10-14**] showed\r\nGPCSputum culture from [**10-14**] grew

In [ ]:
# Extract the text and entities from the annotation
text = annotations[1][0]
entities = annotations[1][1]['entities']

# Generate CoNLL BIO tags
conll_tags = []
prev_entity_end = -1

for start, end, entity_type in entities:
    entity_text = text[start:end]
    entity_tokens = entity_text.split()
    num_tokens = len(entity_tokens)

    if prev_entity_end == start:
        conll_tags.extend(["I-" + entity_type] * num_tokens)
    else:
        conll_tags.extend(["B-" + entity_type] + ["I-" + entity_type] * (num_tokens - 1))

    prev_entity_end = end

# Print the CoNLL BIO tags
print(conll_tags)

['B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-FORM', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'I-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'I-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'I-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'B-FORM', 'B-DURATION', 'I-DURATION', 'B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'B-DOSAGE', 'I-DOSAGE', 'B-FORM', 'B-DURATION', 'I-DURATION', 'I-DURATION', 'I-DURATION', 'B-DRUG', 'I-DRUG', 'B-DOSAGE', 'I-

In [ ]:
# Extract the text and entities from the annotation
text = annotations[1][0]
entities = annotations[1][1]['entities']

# Generate SpaCy training offsets and BILOU tags
spacy_offsets = []
bilou_tags = []

current_offset = 0
for start, end, entity_type in entities:
    entity_text = text[start:end]
    entity_length = len(entity_text)
    entity_start = current_offset + text[current_offset:].index(entity_text)
    entity_end = entity_start + entity_length

    spacy_offsets.append((entity_start, entity_end, entity_type))

    for i in range(entity_length):
        if i == 0:
            bilou_tags.append('B-' + entity_type)
        elif i == entity_length - 1:
            bilou_tags.append('L-' + entity_type)
        else:
            bilou_tags.append('I-' + entity_type)

    current_offset = entity_end

print(spacy_offsets)
print(bilou_tags)

[(2, 7, 'DRUG'), (8, 14, 'DOSAGE'), (15, 21, 'FORM'), (31, 38, 'FORM'), (47, 60, 'DURATION'), (92, 107, 'DRUG'), (108, 118, 'DOSAGE'), (119, 125, 'FORM'), (148, 162, 'DURATION'), (166, 173, 'DRUG'), (174, 179, 'DOSAGE'), (180, 186, 'FORM'), (228, 236, 'DURATION'), (247, 257, 'DRUG'), (258, 264, 'DOSAGE'), (285, 300, 'DURATION'), (312, 326, 'DRUG'), (327, 335, 'DOSAGE'), (336, 341, 'FORM'), (364, 378, 'DURATION'), (382, 399, 'DRUG'), (400, 416, 'DOSAGE'), (429, 436, 'FORM'), (472, 485, 'DURATION'), (520, 532, 'DRUG'), (537, 543, 'FORM'), (564, 572, 'DURATION'), (583, 595, 'DRUG'), (596, 601, 'DOSAGE'), (602, 608, 'FORM'), (629, 637, 'DURATION'), (649, 659, 'DRUG'), (660, 664, 'DOSAGE'), (665, 671, 'FORM'), (692, 700, 'DURATION'), (711, 724, 'DRUG'), (725, 731, 'DOSAGE'), (732, 738, 'FORM'), (768, 781, 'DURATION'), (804, 819, 'DRUG'), (820, 829, 'DOSAGE'), (830, 836, 'FORM'), (869, 882, 'DURATION'), (908, 926, 'DRUG'), (927, 930, 'DOSAGE'), (931, 936, 'FORM'), (982, 995, 'DURATION'), (10

In [ ]:
import spacy
from spacy.training import offsets_to_biluo_tags

nlp = spacy.load("en_core_web_sm")

for i in range(len(annotations)):
  text = annotations[i][0]
# Load a trained SpaCy model
  doc = nlp(text)

# Convert the entities to SpaCy training offsets
offsets = [(start, end, label) for start, end, label in entities]
doc.ents = [doc.char_span(start, end, label=label) for start, end, label in offsets]

# Convert the entities to BIOUL tags
tags = offsets_to_biluo_tags(doc, offsets)

print(tags)

TypeError: ignored

In [ ]:
training_data = []
for i in annotations:
    text = i[0]
    entities = i[1]['entities']

    # Create a SpaCy Doc object from the text
    doc = nlp(text)

    # Convert the entities to SpaCy training offsets
    offsets = [(start, end, label) for start, end, label in entities]
    # Check if the span is valid before adding it to doc.ents
    doc_ents = []
    for start, end, label in offsets:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            doc_ents.append(span)

    # Set the entities in doc.ents
    doc.ents = doc_ents

    # Convert the entities to BIOUL tags
    tags = offsets_to_biluo_tags(doc, offsets)

    # Append the text and BIOUL tags to the training data
    training_data.append((text, {'entities': entities, 'tags': tags}))

# Print the converted training data
for data in training_data:
    print("Text:", data[0])
    print("Entities:", data[1]['entities'])
    print("Tags:", data[1]['tags'])
    print()

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "
Docusate Sodium 100 mg Capsule Sig: One (1) Caps..." with entities "[(2, 17, 'DRUG'), (18, 24, 'DOSAGE'), (25, 32, 'FO...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "
Divalproex (DELayed Release) 250 mg PO QAM
Diva..." with entities "[(2, 12, 'DRUG'), (31, 37, 'DOSAGE'), (46, 56, 'DR...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be ali

Text: 
year old man 1 month s/p AVR, MVR, and CABG x 3, now with
fever to 100.7, WBC 18.7, and erythema at superior aspect of
wound. Pt recently had a long hospitalization following CABG and
MVR/AVR([**Date range (3) 83151**]), which was complicated by post-op
CVA, aflutter/afib with unsuccessful cardioversion, and a
HA-pna. The pt was discharged with a trach to [**Hospital3 **],
due to his inability to manage secretions. The pt did well at
rehab, until [**2125-10-4**], when he developed a low-grade fever and
CXR showed a new left lung base consolidation. He continued to
have temps to 99, and on [**10-14**] pt was started on ceftriaxone 1g
q24h. On [**10-15**] the pt was also started on vancomycin 1g q12h. Per
rehab reports the pt developed confusion and RR in the high
The pt was thought to be in volume overload, and was given
lasix 20mg IV once on [**10-17**]. Gram stain of sputum from [**10-14**] showed
GPCSputum culture from [**10-14**] grew 2+ E. coli that was
pansensitive. Cdiff f

In [ ]:
import csv

# Specify the path to the CSV file
csv_file = "dataset3.csv"

# Save the training data to a CSV file
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Text", "Tags"])  # Write the header row

    for data in training_data:
        text = data[0]
        tags = data[1]['tags']
        writer.writerow([text, " ".join(tags)])  # Write each row with text and tags joined by a space

print("Training data saved to", csv_file)

Training data saved to dataset3.csv


In [1]:
import pandas as pd

df = pd.read_csv('dataset3.csv')
df

,Text,Tags
0,"\r\nyear old man 1 month s/p AVR, MVR, and CAB...",O O O O O O O O O O O O O O O O O O O O O O O ...
1,\r\nSenna 8.6 mg Tablet Sig: 1-2 Tablets PO BI...,O U-DRUG B-DOSAGE L-DOSAGE U-FORM O O O O O U-...
2,\r\nMr. [**Known lastname 19017**] is a 69-yea...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,"\r\nNitroglycerin 0.3 mg Tablet, Sublingual Si...",O U-DRUG B-DOSAGE L-DOSAGE U-FORM O O O O O O ...
4,\r\nYO female with metastatic adenocarcinoma w...,O O O O B-DISEASE L-DISEASE O O O O O O O O O ...
...,...,...
70,"\r\nThe pt is a 78 yo M with a h/o CAD,\r\nHTN...",O O O O O O O O O O O O O O O O O O O O O O O ...
71,"\r\nwith h/o IVDA, cirrhosis [**1-19**] autoim...",O O O O O O O O O O O O O O O O O B-DISEASE L-...
72,\r\nfluconazole 200 mg Tablet Sig: Two (2) Tab...,O U-DRUG B-DOSAGE L-DOSAGE U-FORM O O O O O O ...
73,\r\n[**Known firstname 17**] is a 52-year-old ...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [2]:
df.Tags.nunique()

75

In [3]:
import spacy
from spacy.training import offsets_to_biluo_tags

nlp = spacy.load("en_core_web_sm")

In [4]:

# Initialize lists to store tokens and tags
token_list = []
tag_list = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    text = row['Text']
    tags = row['Tags'].split()  # Split the tags by space to create a list

    # Tokenize the text using spaCy
    doc = nlp(text)

    # Extract the tokens and corresponding tags
    tokens = [token.text for token in doc]
    tags = tags[:len(tokens)]  # Make sure tags align with tokens

    # Append the tokens and tags to the respective lists
    token_list.append(tokens)
    tag_list.append(tags)

# Print the token and tag lists
print(token_list)
print(tag_list)

[['\r\n', 'year', 'old', 'man', '1', 'month', 's', '/', 'p', 'AVR', ',', 'MVR', ',', 'and', 'CABG', 'x', '3', ',', 'now', 'with', '\r\n', 'fever', 'to', '100.7', ',', 'WBC', '18.7', ',', 'and', 'erythema', 'at', 'superior', 'aspect', 'of', '\r\n', 'wound', '.', 'Pt', 'recently', 'had', 'a', 'long', 'hospitalization', 'following', 'CABG', 'and', '\r\n', 'MVR', '/', 'AVR([**Date', 'range', '(', '3', ')', '83151', '*', '*', ']', ')', ',', 'which', 'was', 'complicated', 'by', 'post', '-', 'op', '\r\n', 'CVA', ',', 'aflutter', '/', 'afib', 'with', 'unsuccessful', 'cardioversion', ',', 'and', 'a', '\r\n', 'HA', '-', 'pna', '.', 'The', 'pt', 'was', 'discharged', 'with', 'a', 'trach', 'to', '[', '*', '*', 'Hospital3', '*', '*', ']', ',', '\r\n', 'due', 'to', 'his', 'inability', 'to', 'manage', 'secretions', '.', 'The', 'pt', 'did', 'well', 'at', '\r\n', 'rehab', ',', 'until', '[', '*', '*', '2125', '-', '10', '-', '4', '*', '*', ']', ',', 'when', 'he', 'developed', 'a', 'low', '-', 'grade', 'f

In [5]:
unique = set([tag for tags in tag_list for tag in tags])
tag_to_int = {tag: i for i, tag in enumerate(unique)}

# Convert tags to integers
tag_list_int = [[tag_to_int[tag] for tag in tags] for tags in tag_list]

# Print the tag list with integers
print(tag_list_int)

[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 17, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 17, 21, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 17, 21, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [6]:
tag_to_int

{'B-DISEASE': 0,
 'L-DOSAGE': 1,
 'O': 2,
 'I-DOSAGE': 3,
 'B-DRUG': 4,
 'I-DISEASE': 5,
 'I-DRUG': 6,
 'I-FORM': 7,
 'U-DISEASE': 8,
 'B-FORM': 9,
 'I-DURATION': 10,
 'L-DURATION': 11,
 'L-FORM': 12,
 'U-DOSAGE': 13,
 'L-DISEASE': 14,
 '-': 15,
 'U-DURATION': 16,
 'U-DRUG': 17,
 'L-DRUG': 18,
 'B-DURATION': 19,
 'U-FORM': 20,
 'B-DOSAGE': 21}

In [7]:
import pandas as pd

# Create a DataFrame with 'text' and 'tags' columns
df = pd.DataFrame({'text': token_list, 'tags': tag_list_int})

# Print the DataFrame
df

,text,tags
0,"[\r\n, year, old, man, 1, month, s, /, p, AVR,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,"[\r\n, Senna, 8.6, mg, Tablet, Sig, :, 1, -, 2...","[2, 17, 21, 1, 20, 2, 2, 2, 2, 2, 20, 2, 2, 2,..."
2,"[\r\n, Mr., [, *, *, Known, lastname, 19017, *...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,"[\r\n, Nitroglycerin, 0.3, mg, Tablet, ,, Subl...","[2, 17, 21, 1, 20, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,"[\r\n, YO, female, with, metastatic, adenocarc...","[2, 2, 2, 2, 0, 14, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
...,...,...
70,"[\r\n, The, pt, is, a, 78, yo, M, with, a, h, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
71,"[\r\n, with, h, /, o, IVDA, ,, cirrhosis, [, *...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
72,"[\r\n, fluconazole, 200, mg, Tablet, Sig, :, T...","[2, 17, 21, 1, 20, 2, 2, 2, 2, 2, 2, 20, 2, 2,..."
73,"[\r\n, [, *, *, Known, firstname, 17, *, *, ],...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [8]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
!pip install datasets
from datasets import DatasetDict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.4 MB/s eta 0:00:00


In [10]:
df1= df[:65]
df2 =df[65:]

In [11]:
from datasets import Dataset, DatasetDict

# Create a dictionary with train, validation, and test datasets
dataset_dict = DatasetDict({
    'train': Dataset.from_pandas(df1),
    'validation': Dataset.from_pandas(df2)
})

# Print the dataset dictionary
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'tags'],
        num_rows: 65
    })
    validation: Dataset({
        features: ['text', 'tags'],
        num_rows: 10
    })
})


In [12]:
dataset_dict['train']['text']

[['\r\n',
  'year',
  'old',
  'man',
  '1',
  'month',
  's',
  '/',
  'p',
  'AVR',
  ',',
  'MVR',
  ',',
  'and',
  'CABG',
  'x',
  '3',
  ',',
  'now',
  'with',
  '\r\n',
  'fever',
  'to',
  '100.7',
  ',',
  'WBC',
  '18.7',
  ',',
  'and',
  'erythema',
  'at',
  'superior',
  'aspect',
  'of',
  '\r\n',
  'wound',
  '.',
  'Pt',
  'recently',
  'had',
  'a',
  'long',
  'hospitalization',
  'following',
  'CABG',
  'and',
  '\r\n',
  'MVR',
  '/',
  'AVR([**Date',
  'range',
  '(',
  '3',
  ')',
  '83151',
  '*',
  '*',
  ']',
  ')',
  ',',
  'which',
  'was',
  'complicated',
  'by',
  'post',
  '-',
  'op',
  '\r\n',
  'CVA',
  ',',
  'aflutter',
  '/',
  'afib',
  'with',
  'unsuccessful',
  'cardioversion',
  ',',
  'and',
  'a',
  '\r\n',
  'HA',
  '-',
  'pna',
  '.',
  'The',
  'pt',
  'was',
  'discharged',
  'with',
  'a',
  'trach',
  'to',
  '[',
  '*',
  '*',
  'Hospital3',
  '*',
  '*',
  ']',
  ',',
  '\r\n',
  'due',
  'to',
  'his',
  'inability',
  'to',
  '

In [13]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [14]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.5 MB/s eta 0:00:00


In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [16]:
example = dataset_dict["train"][4]
tokenized_input = tokenizer(example['text'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (585 > 512). Running this sequence through the model will result in indexing errors


['[CLS]', 'yo', 'female', 'with', 'meta', '##static', 'aden', '##oca', '##rc', '##ino', '##ma', 'with', 'unknown', 'primary', 'on', 'c2', '##d', '##1', 'gem', '##cit', '##abi', '##ne', '/', 'ir', '##ino', '##tec', '##an', 'and', 'with', 'mali', '##gnant', 'pl', '##eur', '##al', 'e', '##ff', '##usions', 'presented', 'to', '[', '*', '*', 'hospital', '##1', '18', '*', '*', ']', 'ed', 'with', 'severe', 'r', 'flank', 'pain', ',', 'radiating', 'to', 'chest', '.', 'patient', 'reports', 'pain', 'was', '[', '*', '*', '9', '-', '19', '*', '*', ']', 'in', 'severity', '.', 'she', 'was', 'otherwise', 'as', '##ym', '##pt', '##oma', '##tic', ',', 'denying', 'short', '##ness', 'of', 'breath', 'or', 'coughing', 'at', 'presentation', '.', 'she', 'experienced', 'r', 'flank', 'pain', 'previously', 'for', 'which', 'she', 'had', 'applied', 'a', 'fen', '##tan', '##yl', 'patch', 'with', 'adequate', 'pain', 'control', '.', 'of', 'note', ',', 'she', 'has', 'known', 'l', '##ytic', 'bone', 'lesions', 'to', 'the',

In [17]:
len(example["tags"]), len(tokenized_input["input_ids"])

(504, 585)

In [18]:
print(tokenized_input.word_ids())

[None, 1, 2, 3, 4, 4, 5, 5, 5, 5, 5, 6, 7, 8, 10, 11, 11, 11, 12, 12, 12, 12, 13, 14, 14, 14, 14, 15, 16, 17, 17, 18, 18, 18, 20, 20, 20, 21, 22, 23, 24, 25, 26, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 62, 63, 64, 65, 65, 65, 65, 65, 66, 67, 68, 68, 69, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 91, 91, 92, 93, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 106, 107, 109, 110, 111, 112, 113, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 124, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 140, 141, 142, 143, 144, 145, 145, 146, 147, 149, 150, 151, 152, 154, 154, 154, 155, 156, 156, 156, 157, 157, 157, 158, 159, 160, 161, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 172, 172, 173, 175, 176, 176, 177, 177, 177, 177, 177, 178, 179, 180, 180, 180, 181, 182, 184, 185, 186, 187, 187, 187, 188, 188, 188, 188, 188, 189, 

In [19]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else df['tags'][4][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

585 585


In [20]:
label_all_tokens = True

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenize_and_align_labels(dataset_dict['train'][:4])

{'input_ids': [[101, 2095, 2214, 2158, 1015, 3204, 1055, 1013, 1052, 20704, 2099, 1010, 19842, 2099, 1010, 1998, 9298, 2290, 1060, 1017, 1010, 2085, 2007, 9016, 2000, 2531, 1012, 1021, 1010, 25610, 2278, 2324, 1012, 1021, 1010, 1998, 9413, 26688, 2863, 2012, 6020, 7814, 1997, 6357, 1012, 13866, 3728, 2018, 1037, 2146, 2902, 3989, 2206, 9298, 2290, 1998, 19842, 2099, 1013, 20704, 2099, 1006, 1031, 1008, 1008, 3058, 2846, 1006, 1017, 1007, 6640, 16068, 2487, 1008, 1008, 1033, 1007, 1010, 2029, 2001, 8552, 2011, 2695, 1011, 6728, 26226, 2050, 1010, 10028, 26878, 1013, 28697, 2497, 2007, 7736, 4003, 3695, 27774, 1010, 1998, 1037, 5292, 1011, 1052, 2532, 1012, 1996, 13866, 2001, 14374, 2007, 1037, 19817, 6776, 2000, 1031, 1008, 1008, 2902, 2509, 1008, 1008, 1033, 1010, 2349, 2000, 2010, 13720, 2000, 6133, 3595, 8496, 1012, 1996, 13866, 2106, 2092, 2012, 24497, 1010, 2127, 1031, 1008, 1008, 18164, 2629, 1011, 2184, 1011, 1018, 1008, 1008, 1033, 1010, 2043, 2002, 2764, 1037, 2659, 1011, 3694,

In [23]:
tokenized_datasets_train = dataset_dict['train'].map(tokenize_and_align_labels, batched=True)
tokenized_datasets_valid = dataset_dict['validation'].map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [24]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=22)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [25]:
args = TrainingArguments(
    f"test-ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
from datasets import load_dataset, load_metric

In [28]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=154a3f0a2ed536bf325d3b0d9ecfd945d4bd3ccd239903205afc0df295909c8c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [57]:
metric = load_metric("seqeval")

In [58]:
label_list=['-',
 'B-DISEASE',
 'B-DOSAGE',
 'B-DRUG',
 'B-DURATION',
 'B-FORM',
 'I-DISEASE',
 'I-DOSAGE',
 'I-DRUG',
 'I-DURATION',
 'I-FORM',
 'L-DISEASE',
 'L-DOSAGE',
 'L-DRUG',
 'L-DURATION',
 'L-FORM',
 'O',
 'U-DISEASE',
 'U-DOSAGE',
 'U-DRUG',
 'U-DURATION',
 'U-FORM']

In [59]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [60]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [61]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.773770,0.832394,0.848224,0.840235,0.810047
2,No log,0.679138,0.855477,0.868676,0.862026,0.833999
3,No log,0.623811,0.868477,0.883746,0.876045,0.854624
4,No log,0.596223,0.871822,0.885899,0.878804,0.860279
5,No log,0.584864,0.873896,0.887693,0.880740,0.861610


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: - seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-DURATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-DOSAGE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-FORM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-DURATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_

TrainOutput(global_step=45, training_loss=0.7189635382758246, metrics={'train_runtime': 34.7444, 'train_samples_per_second': 9.354, 'train_steps_per_second': 1.295, 'total_flos': 82027817973108.0, 'train_loss': 0.7189635382758246, 'epoch': 5.0})

In [62]:
trainer.evaluate()

{'eval_loss': 0.5848640203475952,
 'eval_precision': 0.8738961497703992,
 'eval_recall': 0.8876928597057768,
 'eval_f1': 0.8807404770380918,
 'eval_accuracy': 0.8616101131071191,
 'eval_runtime': 0.4977,
 'eval_samples_per_second': 20.094,
 'eval_steps_per_second': 4.019,
 'epoch': 5.0}

In [63]:
model.save_pretrained('ner_model')
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [64]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label:str(i) for i,label in enumerate(label_list)
}

In [65]:
import json

config = json.load(open('ner_model/config.json'))

In [66]:
json.dump(config,open('ner_model/config.json','w'))

In [67]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained('ner_model')
model_fine_tuned

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [68]:
from transformers import pipeline

In [69]:
nlp = pipeline('ner',model=model_fine_tuned,tokenizer=tokenizer)

example=('Donepezil 10 mg PO HS Gabapentin 300 mg PO HS Seizure LeVETiracetam [**2147**] mg PO BID Metoprolol Tartrate 12.5 mg PO BID hold for SBP<100, HR<55 Omeprazole 20 mg PO BID gastritis prophylaxis PredniSONE 40 mg PO DAILY drug reaction Duration: 6 Weeks')

ner_results= nlp(example)
ner_results

[{'entity': 'LABEL_17',
  'score': 0.48799893,
  'index': 1,
  'word': 'done',
  'start': 0,
  'end': 4},
 {'entity': 'LABEL_17',
  'score': 0.6012414,
  'index': 2,
  'word': '##pe',
  'start': 4,
  'end': 6},
 {'entity': 'LABEL_17',
  'score': 0.55802494,
  'index': 3,
  'word': '##zi',
  'start': 6,
  'end': 8},
 {'entity': 'LABEL_17',
  'score': 0.54373676,
  'index': 4,
  'word': '##l',
  'start': 8,
  'end': 9},
 {'entity': 'LABEL_21',
  'score': 0.2574162,
  'index': 5,
  'word': '10',
  'start': 10,
  'end': 12},
 {'entity': 'LABEL_1',
  'score': 0.33036467,
  'index': 6,
  'word': 'mg',
  'start': 13,
  'end': 15},
 {'entity': 'LABEL_2',
  'score': 0.6601204,
  'index': 7,
  'word': 'po',
  'start': 16,
  'end': 18},
 {'entity': 'LABEL_2',
  'score': 0.6567554,
  'index': 8,
  'word': 'hs',
  'start': 19,
  'end': 21},
 {'entity': 'LABEL_17',
  'score': 0.5593377,
  'index': 9,
  'word': 'ga',
  'start': 22,
  'end': 24},
 {'entity': 'LABEL_17',
  'score': 0.6222215,
  'index'

In [72]:
label2id

{'-': '0',
 'B-DISEASE': '1',
 'B-DOSAGE': '2',
 'B-DRUG': '3',
 'B-DURATION': '4',
 'B-FORM': '5',
 'I-DISEASE': '6',
 'I-DOSAGE': '7',
 'I-DRUG': '8',
 'I-DURATION': '9',
 'I-FORM': '10',
 'L-DISEASE': '11',
 'L-DOSAGE': '12',
 'L-DRUG': '13',
 'L-DURATION': '14',
 'L-FORM': '15',
 'O': '16',
 'U-DISEASE': '17',
 'U-DOSAGE': '18',
 'U-DRUG': '19',
 'U-DURATION': '20',
 'U-FORM': '21'}

In [89]:
import spacy
from spacy import displacy

nlp = spacy.blank("en")  # Create a blank spaCy pipeline

# Define the example in spaCy format
example = []
for item in ner_results:
    if item['entity'] != 'LABEL_2':
        entity = {
            "start": item['start'],
            "end": item['end'],
            "label": item['entity']
        }
        example.append({"text": item['word'], "ents": [entity], "title": None})
    else:
        example.append({"text": item['word'], "ents": [], "title": None})


# Render the entities
displacy.render(example, style="ent", manual=True,jupyter=True)

In [91]:
import spacy
from spacy import displacy

nlp = spacy.blank("en")  # Create a blank spaCy pipeline

# Combine subwords with '#' prefix and assign label of the first subword
combined_example = []
current_word = None
current_label = None

for item in ner_results:
    if item['entity'] != 'LABEL_2':
        if item['word'].startswith("##"):
            if current_word is None:
                current_word = item['word'][2:]
                current_label = item['entity']
            else:
                current_word += item['word'][2:]
        else:
            if current_word is not None:
                entity = {
                    "start": item['start'] - len(current_word),
                    "end": item['end'],
                    "label": current_label
                }
                combined_example.append({"text": current_word, "ents": [entity], "title": None})
                current_word = None
                current_label = None

            entity = {
                "start": item['start'],
                "end": item['end'],
                "label": item['entity']
            }
            combined_example.append({"text": item['word'], "ents": [entity], "title": None})

# Process any remaining combined word
if current_word is not None:
    entity = {
        "start": item['end'] - len(current_word),
        "end": item['end'],
        "label": current_label
    }
    combined_example.append({"text": current_word, "ents": [entity], "title": None})

# Render the entities
displacy.render(combined_example, style="ent", manual=True, jupyter=True)
